## unzip

In [ ]:
import zipfile
zip_file = zipfile.ZipFile('Vaihingen_O/ISPRS_semantic_labeling_Vaihingen.zip')
zip_extract = zip_file.extractall('Vaihingen_test')

## zip

In [ ]:
import os
import zipfile
 
def getZipDir(dirpath, outFullName):
    """
    压缩指定文件夹
    :param dirpath: 目标文件夹路径
    :param outFullName: 压缩文件保存路径+xxxx.zip
    :return: 
    """
    zip = zipfile.ZipFile(outFullName, "w", zipfile.ZIP_DEFLATED)
    for path, dirnames, filenames in os.walk(dirpath):
        # 去掉目标跟路径，只对目标文件夹下边的文件及文件夹进行压缩
        fpath = path.replace(dirpath, '')
 
        for filename in filenames:
            zip.write(os.path.join(path, filename), os.path.join(fpath, filename))
    zip.close()
 
 
if __name__ == "__main__":
    getZipDir(dirpath="vaihingen_2",
              outFullName="vaihingen_2.zip")

## count

In [ ]:
import os

path = 'vaihingen/train/images_512'
files = os.listdir(path)
num = len(files)

print('文件数量：')
print(num)
for a in [
            'area1_', 'area11', 'area13', 'area15', 'area17', 'area21',
            'area23', 'area26', 'area28', 'area3_', 'area30', 'area32',
            'area34', 'area37', 'area5_', 'area7_', 'area6_', 'area24', 
            'area35', 'area16', 'area33'
        ]:
    b = 0
    for file in files:
        if a == file[:6]:   # When dataset is Potsdam, [:6]-->[:4]
            b += 1
    print(a)
    print(b)

## remove files and output the number of files

In [ ]:
import shutil
import os

def remove_file(old_path, new_path):
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    
    filelist = os.listdir(old_path)
    
    cout = 0
    
    for file in filelist:
        filename,extension = os.path.splitext(file)
        if extension == '.png':
            cout += 1
            src = os.path.join(old_path, file)
            dst = os.path.join(new_path, file)
            shutil.copy(src,dst)
    print(cout)


In [ ]:
remove_file('vaihingen/train/images_512/','vaihingen_2/images')

## view pre trained models available in timm

In [ ]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

## replace color ((255,255,0) is unable to be seen clearly.)

In [ ]:
from PIL import Image
import os
import numpy as np
import time

def replace_color(img, src_clr, dst_clr):
    ''' 通过矩阵操作颜色替换程序
    @param img: 图像矩阵
    @param src_clr: 需要替换的颜色(r,g,b)
    @param dst_clr: 目标颜色(r,g,b)
    @return 替换后的图像矩阵
    '''
    img_arr = np.asarray(img, dtype=np.double)

    #分离通道
    r_img = img_arr[:,:,0].copy()
    g_img = img_arr[:,:,1].copy()
    b_img = img_arr[:,:,2].copy()

    #编码
    img = r_img * 256 * 256 + g_img * 256 + b_img
    src_color = src_clr[0] * 256 * 256 + src_clr[1] * 256 + src_clr[2]

    #索引并替换颜色
    r_img[img == src_color] = dst_clr[0]
    g_img[img == src_color] = dst_clr[1]
    b_img[img == src_color] = dst_clr[2]
    
    #合并通道
    dst_img = np.array([r_img, g_img, b_img], dtype=np.uint8)
    #将数据转换为图像数据(h,w,c)
    dst_img = dst_img.transpose(1,2,0)
    
    return dst_img

def replace_color_tran(img, src_clr, dst_clr):
    ''' 通过遍历颜色替换程序
    @param img: 图像矩阵
    @param src_clr: 需要替换的颜色(r,g,b)
    @param dst_clr: 目标颜色r,g,b)
    @return 替换后的图像矩阵
    '''
    img_arr = np.asarray(img, dtype=np.double)

    dst_arr = img_arr.copy()
    for i in range(img_arr.shape[1]):	
        for j in range(img_arr.shape[0]):
            if (img_arr[j][i] == src_clr)[0] == True:
                dst_arr[j][i] = dst_clr

    return np.asarray(dst_arr, dtype=np.uint8)


img = 'vai_big_GT/top_mosaic_09cm_area33_noBoundary.tif'
img = Image.open(img).convert('RGB')
res_img = img.copy()
count = 20
matrix_time = 0
trans_time = 0

for i in range(count):
    #print(i)
    start = time.time()
    dst_img = replace_color(img, (255,255,0), (255,204,0))
    end = time.time()
    matrix_time += (end - start)

    # start = time.time()
    # dst_img = replace_color_tran(img, (8,10,51), (255,0,0))
    # end = time.time()
    # trans_time += (end - start)

    res_img = dst_img

res_img = Image.fromarray(res_img)
res_img.save('vai_big_GT/top_mosaic_09cm_area33_noBoundary_alter.tif')

print('矩阵操作花费时间：', matrix_time / count )
#print('遍历操作花费时间：', trans_time / count )
